In [ ]:
import random, sys, time, os, math
import numpy as np
import xml.dom.minidom
import networkx as nx
import matplotlib.pyplot as plt
from itertools import islice

### Data Preparation

In [ ]:
def read_files(x, dict_result):
    """
    Read .csv results files:

    algorithm ID mapping:
    101: CPBPC-div
    105: CPBPC-div-random
    198: CPBPC-del&div
    398: CPBPC-del&div-noCut
    498: CPBPC-del&div-noPH
    598: CPBPC-del&div-noCutPH
    999: Compact-CP
    1000: Compact-MILP
    1001: Heuristic

    Attributes:
    [
    0: ID
    1: topology	
    2: nb_nodes	
    3: nb_links	
    4: nb_demands	
    5: sec_cost	
    6: nb_demands_ratio	
    7: delay_difference_UB	
    8: algo	
    9: UB	
    10: LB	
    11: optGap	
    12: nodes_evaluated	
    13: runtime	
    14: feasibility	
    15: optimality	
    16: thrCallPH
    17: nb_cuts
    18: nb_cols
    19: nb_PHs
    20: nb_lift
    ]
    """

    mapping = {}
    mapping[101] = '../results/CPBPC-div.csv'
    mapping[105] = '../results/CPBPC-div-random.csv'
    mapping[198] = '../results/CPBPC-del&div.csv'
    mapping[398] = '../results/CPBPC-del&div-noCut.csv'
    mapping[498] = '../results/CPBPC-del&div-noPH.csv'
    mapping[598] = '../results/CPBPC-del&div-noCutPH.csv'
    mapping[999] = '../results/Compact-CP.csv'
    mapping[1000] = '../results/Compact-MILP.csv'
    mapping[1001] = '../results/Heuristic.csv'


    filename = mapping[x]

    # ignore the first line
    f = open(filename)
    line = f.readline()
    params = line.split(',')
    dict_result[x] = {}

    # Fill in the dictionary with the results. key: (topo, algo, delayUB), value = [UB, LB, original_gap, nodes, runtime, feasibility, optimality, augmented_gap]

    # topology nb_nodes nb_links nb_demands avg_capacity nb_demands_ratio delay_difference_UB algo UB LB optGap nodes_evaluated runtime feasibility optimality thrCallPH nb_cuts nb_cols nb_PHs nb_lift
    
    line = f.readline()
    params = line.split(',')
    while len(params) >= 21:
        dict_result[x][(params[1], params[5], params[6], params[7], params[8])] = params
        line = f.readline()
        params = line.split(',')
        # print(len(params))

In [ ]:
def regulate_runtime(x, dict_result):
    """
    when the runtime is greater than 1000'000, regulate it to be 1000'000
    """
    if ('', '', '', '', '') in dict_result[x]:
        del dict_result[x][('', '', '', '', '')]
    for key, value in dict_result[x].items():
        # print(key)
        if float(value[13]) > 1000000:
            value[13] = str(1000000)

In [ ]:
def modify_zero_LB(x, dict_result):
    """
    when the LB is zero and runtime is smaller than 1000'000, modify LB to be UB
    """
    for key, value in dict_result[x].items():
        if float(value[13]) < 1000000:
            value[14] = str(1)
            value[15] = str(1)
            if float(value[10]) < 1.0:
                value[10] = value[9]
                value[11] = str(0.0) 

In [ ]:
def get_default_cost(dict_result):
    """ 
    Use the cost obtained from 1001 (greedy heuristic) as the default cost of all instances
    """
    default_cost = {}
    for key, value in dict_result[1001].items():
        default_cost[key] = float(value[9])

    return default_cost

In [ ]:
def find_missing_configuration(x, dict_result):
    """ 
    Find the missing configuration for algo x
    """
    # print(x)
    for topo in ["giul39", "france", "geant", "germany50", "janos-us-ca", "nobel-eu", "norway", "sun", "ta1", "zib54", "Abvt", "Bandcon", "Canerie", "Evolink", "Goodnet", "HurricaneElectric", "Ibm", "Quest", "Rediris", "Xeex", "random_20_3", "random_20_4", "random_20_5", "random_25_3", "random_25_4", "random_25_5", "random_30_3", "random_30_4", "random_30_5"]:
        for i in ["0", "1"]:
            for j in ["1", "2", "3"]:
                for k in ["1", "2", "3"]:
                    if (topo, i, j, k, str(x)) not in dict_result[x].keys():
                        print(topo, i, j, k, x)


In [ ]:
def replace_by_default(x, dict_result, default_cost):
    """
    In case of no feasible solution found,
    replace 10^15 by default cost, and
    also, re-calculate the corresponding optGap
    """
    for key, value in dict_result[x].items():
        if float(value[9]) >= 1000000000000000:
            key_copy = (key[0], key[1], key[2], key[3], '1001')
            value[9] = str(default_cost[key_copy])
            LB =  float(value[10])
            value[11] = str(default_cost[key_copy] / LB - 1.0)

In [ ]:
algo_ID = [101, 105, 198, 398, 498, 598, 999, 1000, 1001]
dict_result = {}
for ID in algo_ID:
    print(ID)
    read_files(ID, dict_result)
    regulate_runtime(ID, dict_result)
    modify_zero_LB(ID, dict_result)
    find_missing_configuration(ID, dict_result)
default_cost = get_default_cost(dict_result)
for ID in algo_ID:
    if ID != 1001:
        replace_by_default(ID, dict_result, default_cost)

"""
Read .csv results files:
"cp_results_{x}_1000s.csv" where x = {101, 201, 105, 106, 206, 110, 197, 297, 397, 497, 597, 198, 298, 398, 498, 598, 1000, 1001} 
Attributes:
[
0: ID
1: topology	
2: nb_nodes	
3: nb_links	
4: nb_demands	
5: avg_capacity	
6: nb_demands_ratio	
7: delay_difference_UB	
8: algo	
9: UB	
10: LB	
11: optGap	
12: nodes_evaluated	
13: runtime	
14: feasibility	
15: optimality	
16: thrCallPH
17: nb_cuts
18: nb_cols
19: nb_PHs
20: nb_lift
]
"""

In [ ]:
topology_all = ["giul39", "france", "geant", "germany50", "janos-us-ca", "nobel-eu", "norway", "sun", "ta1", "zib54", "Abvt", "Bandcon", "Canerie", "Evolink", "Goodnet", "HurricaneElectric", "Ibm", "Quest", "Rediris", "Xeex", "random_20_3", "random_20_4", "random_20_5", "random_25_3", "random_25_4", "random_25_5", "random_30_3", "random_30_4", "random_30_5"]

### Table 3

In [ ]:
algo_ID = [101, 105, 198, 398, 498, 598, 999, 1000]
topology_included = ["giul39", "france", "geant", "germany50", "janos-us-ca", "nobel-eu", "norway", "sun", "ta1", "zib54", "Abvt", "Bandcon", "Canerie", "Evolink", "Goodnet", "HurricaneElectric", "Ibm", "Quest", "Rediris", "Xeex", "random_20_3", "random_20_4", "random_20_5", "random_25_3", "random_25_4", "random_25_5", "random_30_3", "random_30_4", "random_30_5"]
gap = {}
gap_ct = {}
nodes = {}
nodes_ct = {}
all_nodes = {}
all_nodes_ct = {}
feasibility = {}
columns = {}
columns_ct = {}
runtime = {}
runtime_ct = {}
opt_runtime = {}
opt_runtime_ct = {}
cuts = {}
PHs = {}
lifts = {}
all_columns = {}
all_columns_ct = {}
for ID in algo_ID:
    print(ID)
    gap[ID] = 0.0
    gap_ct[ID] = 0
    nodes[ID] = 0.0
    nodes_ct[ID] = 0
    all_nodes[ID] = 0.0
    all_nodes_ct[ID] = 0
    feasibility[ID] = 0
    columns[ID] = 0
    columns_ct[ID] = 0
    all_columns[ID] = 0
    all_columns_ct[ID] = 0
    runtime[ID] = 0.0
    runtime_ct[ID] = 0
    opt_runtime[ID] = 0.0
    opt_runtime_ct[ID] = 0
    cuts[ID] = 0
    PHs[ID] = 0
    lifts[ID] = 0
    for key, value in dict_result[ID].items():
        # if value[15] != "1":
        #     gap[ID] += float(value[11])
        #     gap_ct[ID] += 1
        #     if float(value[11]) > 1.0:
        #         print(key)
        if key[0] in topology_included:
            # if ID == 999:
            #     print(key)
            gap[ID] += float(value[11])
            gap_ct[ID] += 1
            runtime[ID] += float(value[13])
            runtime_ct[ID] += 1
            all_nodes[ID] += float(value[12])
            all_nodes_ct[ID] += 1
            cuts[ID] += float(value[17])
            PHs[ID] += float(value[19])
            lifts[ID] += float(value[20])
            all_columns[ID] += float(value[18])
            all_columns_ct[ID] += 1
            if value[15] == "1": 
                nodes[ID] += float(value[12])
                nodes_ct[ID] += 1
                columns[ID] += float(value[18])
                columns_ct[ID] += 1
                opt_runtime[ID] += float(value[13])
                opt_runtime_ct[ID] += 1
            if value[14] == "1":
                feasibility[ID] += 1

    gap[ID] /= gap_ct[ID]
    print("mean opt gap: ", gap[ID], "number of instances", gap_ct[ID])
    nodes[ID] /= nodes_ct[ID]
    print("mean evaluated nodes over optimal instances: ", nodes[ID], "number of optimal instances", nodes_ct[ID])
    all_nodes[ID] /= all_nodes_ct[ID]
    print("mean evaluated nodes over all instances: ", all_nodes[ID], "number of all instances", all_nodes_ct[ID])
    print("number of feasible instances: ", feasibility[ID])
    columns[ID] /= columns_ct[ID]
    print("number of columns of all optimal instances: ", columns[ID])
    all_columns[ID] /= all_columns_ct[ID]
    print("number of columns of all instances: ", all_columns[ID])
    runtime[ID] /= runtime_ct[ID]
    print("mean runtime over all instances: ", runtime[ID])
    opt_runtime[ID] /= opt_runtime_ct[ID]
    print("mean runtime over all optimal instances: ", opt_runtime[ID])
    cuts[ID] /= all_nodes_ct[ID]
    print("mean number of cuts over all instances: ", cuts[ID])
    PHs[ID] /= all_nodes_ct[ID]
    print("mean number of PHs over all instances: ", PHs[ID])
    lifts[ID] /= all_nodes_ct[ID]
    print("mean number of lifts over all instances: ", lifts[ID])

### Table 4

In [ ]:
algo_ID = [101, 105, 198, 398, 498, 598, 999, 1000]
topology_included = ["giul39", "france", "geant", "germany50", "janos-us-ca", "nobel-eu", "norway", "sun", "ta1", "zib54"]
gap = {}
gap_ct = {}
nodes = {}
nodes_ct = {}
all_nodes = {}
all_nodes_ct = {}
feasibility = {}
columns = {}
columns_ct = {}
runtime = {}
runtime_ct = {}
opt_runtime = {}
opt_runtime_ct = {}
cuts = {}
PHs = {}
lifts = {}
all_columns = {}
all_columns_ct = {}
for ID in algo_ID:
    print(ID)
    gap[ID] = 0.0
    gap_ct[ID] = 0
    nodes[ID] = 0.0
    nodes_ct[ID] = 0
    all_nodes[ID] = 0.0
    all_nodes_ct[ID] = 0
    feasibility[ID] = 0
    columns[ID] = 0
    columns_ct[ID] = 0
    all_columns[ID] = 0
    all_columns_ct[ID] = 0
    runtime[ID] = 0.0
    runtime_ct[ID] = 0
    opt_runtime[ID] = 0.0
    opt_runtime_ct[ID] = 0
    cuts[ID] = 0
    PHs[ID] = 0
    lifts[ID] = 0
    for key, value in dict_result[ID].items():
        # if value[15] != "1":
        #     gap[ID] += float(value[11])
        #     gap_ct[ID] += 1
        #     if float(value[11]) > 1.0:
        #         print(key)
        if key[0] in topology_included:
            # if ID == 999:
            #     print(key)
            gap[ID] += float(value[11])
            gap_ct[ID] += 1
            runtime[ID] += float(value[13])
            runtime_ct[ID] += 1
            all_nodes[ID] += float(value[12])
            all_nodes_ct[ID] += 1
            cuts[ID] += float(value[17])
            PHs[ID] += float(value[19])
            lifts[ID] += float(value[20])
            all_columns[ID] += float(value[18])
            all_columns_ct[ID] += 1
            if value[15] == "1": 
                nodes[ID] += float(value[12])
                nodes_ct[ID] += 1
                columns[ID] += float(value[18])
                columns_ct[ID] += 1
                opt_runtime[ID] += float(value[13])
                opt_runtime_ct[ID] += 1
            if value[14] == "1":
                feasibility[ID] += 1

    gap[ID] /= gap_ct[ID]
    print("mean opt gap: ", gap[ID], "number of instances", gap_ct[ID])
    nodes[ID] /= nodes_ct[ID]
    print("mean evaluated nodes over optimal instances: ", nodes[ID], "number of optimal instances", nodes_ct[ID])
    all_nodes[ID] /= all_nodes_ct[ID]
    print("mean evaluated nodes over all instances: ", all_nodes[ID], "number of all instances", all_nodes_ct[ID])
    print("number of feasible instances: ", feasibility[ID])
    columns[ID] /= columns_ct[ID]
    print("number of columns of all optimal instances: ", columns[ID])
    all_columns[ID] /= all_columns_ct[ID]
    print("number of columns of all instances: ", all_columns[ID])
    runtime[ID] /= runtime_ct[ID]
    print("mean runtime over all instances: ", runtime[ID])
    opt_runtime[ID] /= opt_runtime_ct[ID]
    print("mean runtime over all optimal instances: ", opt_runtime[ID])
    cuts[ID] /= all_nodes_ct[ID]
    print("mean number of cuts over all instances: ", cuts[ID])
    PHs[ID] /= all_nodes_ct[ID]
    print("mean number of PHs over all instances: ", PHs[ID])
    lifts[ID] /= all_nodes_ct[ID]
    print("mean number of lifts over all instances: ", lifts[ID])

In [ ]:
algo_ID = [101, 105, 198, 398, 498, 598, 999, 1000]
topology_included = ["Abvt", "Bandcon", "Canerie", "Evolink", "Goodnet", "HurricaneElectric", "Ibm", "Quest", "Rediris", "Xeex"]
gap = {}
gap_ct = {}
nodes = {}
nodes_ct = {}
all_nodes = {}
all_nodes_ct = {}
feasibility = {}
columns = {}
columns_ct = {}
runtime = {}
runtime_ct = {}
opt_runtime = {}
opt_runtime_ct = {}
cuts = {}
PHs = {}
lifts = {}
all_columns = {}
all_columns_ct = {}
for ID in algo_ID:
    print(ID)
    gap[ID] = 0.0
    gap_ct[ID] = 0
    nodes[ID] = 0.0
    nodes_ct[ID] = 0
    all_nodes[ID] = 0.0
    all_nodes_ct[ID] = 0
    feasibility[ID] = 0
    columns[ID] = 0
    columns_ct[ID] = 0
    all_columns[ID] = 0
    all_columns_ct[ID] = 0
    runtime[ID] = 0.0
    runtime_ct[ID] = 0
    opt_runtime[ID] = 0.0
    opt_runtime_ct[ID] = 0
    cuts[ID] = 0
    PHs[ID] = 0
    lifts[ID] = 0
    for key, value in dict_result[ID].items():
        # if value[15] != "1":
        #     gap[ID] += float(value[11])
        #     gap_ct[ID] += 1
        #     if float(value[11]) > 1.0:
        #         print(key)
        if key[0] in topology_included:
            gap[ID] += float(value[11])
            gap_ct[ID] += 1
            runtime[ID] += float(value[13])
            runtime_ct[ID] += 1
            all_nodes[ID] += float(value[12])
            all_nodes_ct[ID] += 1
            cuts[ID] += float(value[17])
            PHs[ID] += float(value[19])
            lifts[ID] += float(value[20])
            all_columns[ID] += float(value[18])
            all_columns_ct[ID] += 1
            if value[15] == "1": 
                nodes[ID] += float(value[12])
                nodes_ct[ID] += 1
                columns[ID] += float(value[18])
                columns_ct[ID] += 1
                opt_runtime[ID] += float(value[13])
                opt_runtime_ct[ID] += 1
            if value[14] == "1":
                feasibility[ID] += 1

    gap[ID] /= gap_ct[ID]
    print("mean opt gap: ", gap[ID], "number of instances", gap_ct[ID])
    nodes[ID] /= nodes_ct[ID]
    print("mean evaluated nodes over optimal instances: ", nodes[ID], "number of optimal instances", nodes_ct[ID])
    all_nodes[ID] /= all_nodes_ct[ID]
    print("mean evaluated nodes over all instances: ", all_nodes[ID], "number of all instances", all_nodes_ct[ID])
    print("number of feasible instances: ", feasibility[ID])
    columns[ID] /= columns_ct[ID]
    print("number of columns of all optimal instances: ", columns[ID])
    all_columns[ID] /= all_columns_ct[ID]
    print("number of columns of all instances: ", all_columns[ID])
    runtime[ID] /= runtime_ct[ID]
    print("mean runtime over all instances: ", runtime[ID])
    opt_runtime[ID] /= opt_runtime_ct[ID]
    print("mean runtime over all optimal instances: ", opt_runtime[ID])
    cuts[ID] /= all_nodes_ct[ID]
    print("mean number of cuts over all instances: ", cuts[ID])
    PHs[ID] /= all_nodes_ct[ID]
    print("mean number of PHs over all instances: ", PHs[ID])
    lifts[ID] /= all_nodes_ct[ID]
    print("mean number of lifts over all instances: ", lifts[ID])

In [ ]:
algo_ID = [101, 105, 198, 398, 498, 598, 999, 1000]
topology_included = ["random_20_3", "random_20_4", "random_20_5", "random_25_3", "random_25_4", "random_25_5", "random_30_3", "random_30_4", "random_30_5"]
gap = {}
gap_ct = {}
nodes = {}
nodes_ct = {}
all_nodes = {}
all_nodes_ct = {}
feasibility = {}
columns = {}
columns_ct = {}
runtime = {}
runtime_ct = {}
opt_runtime = {}
opt_runtime_ct = {}
cuts = {}
PHs = {}
lifts = {}
all_columns = {}
all_columns_ct = {}
for ID in algo_ID:
    print(ID)
    gap[ID] = 0.0
    gap_ct[ID] = 0
    nodes[ID] = 0.0
    nodes_ct[ID] = 0
    all_nodes[ID] = 0.0
    all_nodes_ct[ID] = 0
    feasibility[ID] = 0
    columns[ID] = 0
    columns_ct[ID] = 0
    all_columns[ID] = 0
    all_columns_ct[ID] = 0
    runtime[ID] = 0.0
    runtime_ct[ID] = 0
    opt_runtime[ID] = 0.0
    opt_runtime_ct[ID] = 0
    cuts[ID] = 0
    PHs[ID] = 0
    lifts[ID] = 0
    for key, value in dict_result[ID].items():
        # if value[15] != "1":
        #     gap[ID] += float(value[11])
        #     gap_ct[ID] += 1
        #     if float(value[11]) > 1.0:
        #         print(key)
        if key[0] in topology_included:
            gap[ID] += float(value[11])
            gap_ct[ID] += 1
            runtime[ID] += float(value[13])
            runtime_ct[ID] += 1
            all_nodes[ID] += float(value[12])
            all_nodes_ct[ID] += 1
            cuts[ID] += float(value[17])
            PHs[ID] += float(value[19])
            lifts[ID] += float(value[20])
            all_columns[ID] += float(value[18])
            all_columns_ct[ID] += 1
            if value[15] == "1": 
                nodes[ID] += float(value[12])
                nodes_ct[ID] += 1
                columns[ID] += float(value[18])
                columns_ct[ID] += 1
                opt_runtime[ID] += float(value[13])
                opt_runtime_ct[ID] += 1
            if value[14] == "1":
                feasibility[ID] += 1

    gap[ID] /= gap_ct[ID]
    print("mean opt gap: ", gap[ID], "number of instances", gap_ct[ID])
    nodes[ID] /= nodes_ct[ID]
    print("mean evaluated nodes over optimal instances: ", nodes[ID], "number of optimal instances", nodes_ct[ID])
    all_nodes[ID] /= all_nodes_ct[ID]
    print("mean evaluated nodes over all instances: ", all_nodes[ID], "number of all instances", all_nodes_ct[ID])
    print("number of feasible instances: ", feasibility[ID])
    columns[ID] /= columns_ct[ID]
    print("number of columns of all optimal instances: ", columns[ID])
    all_columns[ID] /= all_columns_ct[ID]
    print("number of columns of all instances: ", all_columns[ID])
    runtime[ID] /= runtime_ct[ID]
    print("mean runtime over all instances: ", runtime[ID])
    opt_runtime[ID] /= opt_runtime_ct[ID]
    print("mean runtime over all optimal instances: ", opt_runtime[ID])
    cuts[ID] /= all_nodes_ct[ID]
    print("mean number of cuts over all instances: ", cuts[ID])
    PHs[ID] /= all_nodes_ct[ID]
    print("mean number of PHs over all instances: ", PHs[ID])
    lifts[ID] /= all_nodes_ct[ID]
    print("mean number of lifts over all instances: ", lifts[ID])

### Table 5

In [ ]:
ID = 1000
baseline_UB = {}
baseline_LB = {}
for key, value in dict_result[ID].items():
    if key[2] == '3':
        baseline_UB[key] = float(value[9])
        baseline_LB[key] = float(value[10])
diff_UB_1 = 0.0
diff_UB_2 = 0.0
diff_UB_3 = 0.0
diff_LB_1 = 0.0
diff_LB_2 = 0.0
diff_LB_3 = 0.0
diff_ct = 522/3
oct_1 = 0
oct_2 = 0
oct_3 = 0
fct_1 = 0
fct_2 = 0
fct_3 = 0
rt_1 = 0.0
rt_2 = 0.0
rt_3 = 0.0
gap_1 = 0.0
gap_2 = 0.0
gap_3 = 0.0
for key, value in dict_result[ID].items():
    if key[2] == '3':
        diff_UB_3 += float(value[9])/float(baseline_LB[key])
        diff_LB_3 += float(value[10])/float(baseline_LB[key])
        if value[15] == '1':
            oct_3 += 1
        if value[14] == '1':
            fct_3 += 1
        rt_3 += float(value[13])
        gap_3 += float(value[11])
        key_2 = (key[0], key[1], '2', key[3], key[4])
        diff_UB_2 += float(dict_result[ID][key_2][9])/float(baseline_LB[key])
        diff_LB_2 += float(dict_result[ID][key_2][10])/float(baseline_LB[key])
        if dict_result[ID][key_2][15] == '1':
            oct_2 += 1
        if dict_result[ID][key_2][14] == '1':
            fct_2 += 1
        rt_2 += float(dict_result[ID][key_2][13])
        gap_2 += float(dict_result[ID][key_2][11])
        key_1 = (key[0], key[1], '1', key[3], key[4])
        diff_UB_1 += float(dict_result[ID][key_1][9])/float(baseline_LB[key])
        diff_LB_1 += float(dict_result[ID][key_1][10])/float(baseline_LB[key])
        if dict_result[ID][key_1][15] == '1':
            oct_1 += 1
        if dict_result[ID][key_1][14] == '1':
            fct_1 += 1
        rt_1 += float(dict_result[ID][key_1][13])
        gap_1 += float(dict_result[ID][key_1][11])
diff_UB_1 /= diff_ct
diff_LB_1 /= diff_ct
diff_UB_2 /= diff_ct
diff_LB_2 /= diff_ct
diff_UB_3 /= diff_ct
diff_LB_3 /= diff_ct
rt_1 /= diff_ct
rt_2 /= diff_ct
rt_3 /= diff_ct
gap_1 /= diff_ct
gap_2 /= diff_ct
gap_3 /= diff_ct
print(diff_UB_1, diff_UB_2, diff_UB_3)
print(diff_LB_1, diff_LB_2, diff_LB_3)
print(fct_1, fct_2, fct_3)
print(oct_1, oct_2, oct_3)
print(rt_1, rt_2, rt_3)
print(gap_1, gap_2, gap_3)

In [ ]:
# Focus on 999 now
# compare different delay diff UB
# AI, OR, IP, OS, Network Measurement, Network Calculus, TSDB
ID = 999
diff_UB_1 = 0.0
diff_UB_2 = 0.0
diff_UB_3 = 0.0
diff_LB_1 = 0.0
diff_LB_2 = 0.0
diff_LB_3 = 0.0
diff_ct = len(dict_result[ID])/2
oct_1 = 0
oct_2 = 0
oct_3 = 0
fct_1 = 0
fct_2 = 0
fct_3 = 0
rt_1 = 0.0
rt_2 = 0.0
rt_3 = 0.0
gap_1 = 0.0
gap_2 = 0.0
gap_3 = 0.0
for key, value in dict_result[ID].items():
    if key[1] == '0':
        if value[15] == '1':
            oct_3 += 1
        if value[14] == '1':
            fct_3 += 1
        rt_3 += float(value[13])
        gap_3 += float(value[11])
    if key[1] == '1':
        if value[15] == '1':
            oct_2 += 1
        if value[14] == '1':
            fct_2 += 1
        rt_2 += float(value[13])
        gap_2 += float(value[11])
rt_1 /= diff_ct
rt_2 /= diff_ct
rt_3 /= diff_ct
gap_1 /= diff_ct
gap_2 /= diff_ct
gap_3 /= diff_ct
print(fct_1, fct_2, fct_3)
print(oct_1, oct_2, oct_3)
print(rt_1, rt_2, rt_3)
print(gap_1, gap_2, gap_3)

### Table 6

In [ ]:
# Focus on 101 now
# compare different delay diff UB
# AI, OR, IP, OS, Network Measurement, Network Calculus, TSDB
ID = 1000
diff_UB_1 = 0.0
diff_UB_2 = 0.0
diff_UB_3 = 0.0
diff_LB_1 = 0.0
diff_LB_2 = 0.0
diff_LB_3 = 0.0
diff_ct = len(dict_result[ID])/2
oct_1 = 0
oct_2 = 0
oct_3 = 0
fct_1 = 0
fct_2 = 0
fct_3 = 0
rt_1 = 0.0
rt_2 = 0.0
rt_3 = 0.0
gap_1 = 0.0
gap_2 = 0.0
gap_3 = 0.0
for key, value in dict_result[ID].items():
    if key[1] == '0':
        if value[15] == '1':
            oct_3 += 1
        if value[14] == '1':
            fct_3 += 1
        rt_3 += float(value[13])
        gap_3 += float(value[11])
    if key[1] == '1':
        if value[15] == '1':
            oct_2 += 1
        if value[14] == '1':
            fct_2 += 1
        rt_2 += float(value[13])
        gap_2 += float(value[11])
rt_1 /= diff_ct
rt_2 /= diff_ct
rt_3 /= diff_ct
gap_1 /= diff_ct
gap_2 /= diff_ct
gap_3 /= diff_ct
print(fct_1, fct_2, fct_3)
print(oct_1, oct_2, oct_3)
print(rt_1, rt_2, rt_3)
print(gap_1, gap_2, gap_3)